# **Pandas的索引index的用途**
#### 把数据存储于普通的columns列也能用于数据查询，那使用index有说明好处？
#### index的用途总结：
#### 1.更方便的数据查询；
#### 2.使用index可以获得性能提升
#### 3.自动的数据对齐功能；
#### 4.更多更强大的数据结构支持。

In [14]:
import pandas as pd

In [15]:
df = pd.read_csv("./datas/ml-latest-small/ratings.csv")

In [16]:
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [17]:
df.count()

userId       16
movieId      16
rating       16
timestamp    16
dtype: int64

## 1、使用index查询数据

In [18]:
# 可以使用df.set_index()来修改索引
# drop==False，让索引列还保持在columns
# 比如，我们用原来的"userId"列作为索引了，如果没有drop=False，dataframe中的"userId"这一列会被删除掉
df.set_index("userId",inplace=True,drop=False)

In [19]:
df.head()

,userId,movieId,rating,timestamp
userId,,,,
1,1,1,4.0,964982703
1,1,3,4.0,964981247
1,1,6,4.0,964982224
1,1,47,5.0,964983815
1,1,50,5.0,964982931


In [20]:
df.index

Index([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 67, 129, 160, 167, 536], dtype='int64', name='userId')

In [21]:
# 使用columns的condition查询方法（代码更繁复一点）
df.loc[df["userId"]==1].head()

,userId,movieId,rating,timestamp
userId,,,,
1,1,1,4.0,964982703
1,1,3,4.0,964981247
1,1,6,4.0,964982224
1,1,47,5.0,964983815
1,1,50,5.0,964982931


In [22]:
# 使用index的查询方法（代码更简洁,和上面那个代码的运行结果是一样的）
df.loc[1].head()

,userId,movieId,rating,timestamp
userId,,,,
1,1,1,4.0,964982703
1,1,3,4.0,964981247
1,1,6,4.0,964982224
1,1,47,5.0,964983815
1,1,50,5.0,964982931


## 2、使用index会提升查询性能
#### - 如果index是唯一的，Pandas会使用哈希表优化，查询性能为O(1)；
#### - 如果index不是唯一的，但是有序，Pandas会使用二分查找算法，查询性能为O(logN)；
#### - 如果index是完全随机的，那么每次查询都要扫描全表，查询性能为O(N)；

#### **实验1：完全随机的顺序查询**

In [23]:
# 将数据随机打散
from sklearn.utils import shuffle # 从sklearn.utils(Scikit-learn库的工具模板)导入shuffle函数
df_shuffle = shuffle(df) # 调用shuffle函数，传入数据框df，将df的行数据随机打乱，把打乱后的数据框赋值给df_shuffle

In [27]:
df_shuffle.head()
# 如下结果所示，userId是完全没有顺序的

,userId,movieId,rating,timestamp
userId,,,,
1,1,1,4.0,964982703
160,160,2340,1.0,985383314
536,536,276,3.0,832839990
67,67,5952,2.0,1501274082
1,1,47,5.0,964983815


In [28]:
# 索引是否是递增的
df_shuffle.index.is_monotonic_increasing

False

In [29]:
df_shuffle.index.is_unique

False

In [32]:
# 计时，查询id==67数据性能
%timeit df_shuffle.loc[67]
# %timeit是Python里Jupyter Notebook的魔法命令，专门用来测试代码运行时间。
# %timeit：魔法命令前缀，告诉环境“接下来要测试这段代码的耗时”，是IPython等交互环境特有的便携功能。（平均性能）

138 μs ± 2.13 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


#### **实验2：将index排序后的查询**

In [33]:
df_sorted = df_shuffle.sort_index()

In [34]:
df_sorted.head()

,userId,movieId,rating,timestamp
userId,,,,
1,1,1,4.0,964982703
1,1,47,5.0,964983815
1,1,157,5.0,964984100
1,1,50,5.0,964982931
1,1,3,4.0,964981247


In [35]:
# 索引是否是递增的
df_sorted.index.is_monotonic_increasing

True

In [36]:
df_sorted.index.is_unique

False

In [37]:
%timeit df_sorted.loc[67]

124 μs ± 2.53 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## 3、使用index能自动对其数据
#### 包括series和dataframe

In [38]:
s1 = pd.Series([1,2,3],index=list("abc"))

In [39]:
s1

a    1
b    2
c    3
dtype: int64

In [40]:
s2 = pd.Series([2,3,4],index=list("bcd"))

In [41]:
s2

b    2
c    3
d    4
dtype: int64

In [42]:
s1+s2

a    NaN
b    4.0
c    6.0
d    NaN
dtype: float64

## 4、使用index更多更强大的数据结构支持
#### *很多强大的索引数据结构：*
#### - CategoricallIndex，基于分类数据的Index，提升性能；
#### - MultiIndex，多维索引，用于groupby多维聚合后结果等；
#### - DatetimeIndex，时间类型索引，强大的日期和时间的方法支持；